In [ ]:
import numpy as np
from scipy.optimize import minimize
from scipy.interpolate import RegularGridInterpolator

# Parameters
gamma = 2.0  # Risk aversion
beta = 0.96  # Discount factor
delta = 0.1  # Depreciation rate
r = 0.05     # Interest rate
K_max = 10   # Maximum capital
D_max = 10   # Maximum debt

# Utility function
def utility(C):
    if C <= 0:
        return -np.inf
    return (C**(1 - gamma)) / (1 - gamma)

# Production function
def production(K):
    return K ** 0.5

# State space grids
K_grid = np.linspace(1, K_max, 50)  # Physical capital grid
D_grid = np.linspace(0, D_max, 50)  # Debt grid

# Value function initialization
V = np.zeros((len(K_grid), len(D_grid)))

# Interpolation function
def interp_value_function(V):
    return RegularGridInterpolator((K_grid, D_grid), V, bounds_error=False, fill_value=None)

# Value function iteration
tol = 1e-4
diff = 1
max_iter = 1000
iter_count = 0

while diff > tol and iter_count < max_iter:
    V_new = np.copy(V)
    interp_func = interp_value_function(V_new)
    for i, K in enumerate(K_grid):
        for j, D in enumerate(D_grid):
            Y = production(K) - r * D
            obj = lambda x: -(utility(x[0]) + beta * interp_func(np.array([[max((1 - delta) * K + x[1] - x[0], 0), max((1 + r) * D - x[0], 0)]]))[0])
            bounds = [(0, Y), (0, max(Y, 0))]  # Ensure bounds are not negative
            res = minimize(obj, [Y / 2, 0], bounds=bounds, method='trust-constr')
            V_new[i, j] = -res.fun if res.success else V[i, j]

    diff = np.linalg.norm(V_new - V)
    V = V_new
    iter_count += 1
    print(f'Iteration {iter_count}, diff = {diff}')

# Final value function
print("Final value function:")
print(V)

C:\Users\chans\anaconda3\lib\site-packages\scipy\optimize\_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '
